In [1]:
import pandas as pd
import statsmodels.api as sm
import numpy as np

I need to add the possibility to add covariates to the untreated twfe--- check
I need to make sure I dont drop the treated but just separate them --- check
I need to construct the function that creates the aggrete parameters using proper weights, create a weights function for this purpose.
I need to make sure the graph for the hours over days is included in the data set 
test the boot error function, and make sure I get the diagonal of the covariance matrix.
I need to create another function that make the testing of parallel trends on the untreated

In [2]:
def compute_treatment_effects(data, outcome, treatment, unit_id, time_id, covariates=[]):
    # Prepare the variables for fixed-effects regression
    features = [outcome, treatment] + covariates
    
    # Dummy encode first and then split into treated and untreated
    full_data = pd.get_dummies(data, columns=[unit_id, time_id], drop_first=True)
    
    # Convert Boolean to Integer
    for col in full_data.select_dtypes(include=['bool']).columns:
        full_data[col] = full_data[col].astype(int)
    
    # Identify and skip truly textual columns
    skip_cols = []
    for col in full_data.select_dtypes(include=['object']).columns:
        if col not in features:
            try:
                full_data[col] = pd.to_numeric(full_data[col])
            except ValueError:
                print(f"Skipping column {col} as it appears to be truly textual.")
                skip_cols.append(col)
    
    full_data = full_data.drop(skip_cols, axis=1)
    
    # Split into treated and untreated groups
    data_untreated = full_data[full_data[treatment] == 0]
    data_treated = full_data[full_data[treatment] == 1]
    
    # Check for untreated units
    if len(data_untreated) == 0:
        raise ValueError("There are no untreated units for comparison.")
    
    # Create feature matrix and target for untreated units
    X = data_untreated.drop(features, axis=1)
    y = data_untreated[outcome]

    # Debug statements to understand data types and values
    # print("Debug Info: Data types of X columns:")
    # print(X.dtypes)
    # print("Debug Info: Data types of y:")
    # print(y.dtypes)
    
    # Run the two-way fixed effects regression on untreated units
    model = sm.OLS(y, sm.add_constant(X))
    results = model.fit()
    
    # Prepare the feature matrix for treated units
    X_treated = data_treated.drop(features, axis=1)
    
    # Add any missing columns to X_treated and fill with zeros
    for col in X.columns:
        if col not in X_treated.columns:
            X_treated[col] = 0
            
    X_treated = X_treated[X.columns]  # Reorder columns to match X
    
    # Compute predicted outcome for treated units
    data.loc[data[treatment] == 1, 'y_0'] = results.predict(sm.add_constant(X_treated))
    
    # Compute treatment effects
    data['t_effects'] = data[outcome] - data['y_0']
    
    return data

In [3]:
# Number of units, time periods, and cohorts
n_units = 10
n_time = 10
n_cohorts = 3

# Random seed for reproducibility
np.random.seed(42)

# Generate a DataFrame
df = pd.DataFrame({
    'unit_id': np.repeat(range(1, n_units + 1), n_time),
    'time_id': list(range(1, n_time + 1)) * n_units,
    'treatment': np.random.choice([0, 1], n_units * n_time),
    'covariate1': np.random.normal(0, 1, n_units * n_time),
    'covariate2': np.random.normal(0, 1, n_units * n_time),
    'cohort': np.random.choice([2010, 2011, 2012], n_units * n_time)
})

# Adding a few never-treated units
df.loc[df['unit_id'].isin([1, 2]), 'treatment'] = 0

# Simulating potential outcomes under the control (Y0)
df['Y0'] = 5 + 0.5 * df['covariate1'] + 0.3 * df['covariate2'] + 0.2 * df['unit_id'] + 0.1 * df['time_id'] + np.random.normal(0, 1, n_units * n_time)

# Defining treatment effect (constant for all units as 2)
df['treatment_effect'] = 2

# Simulating potential outcomes under the treatment (Y1)
df['Y1'] = df['Y0'] + df['treatment_effect']

# Constructing the observed outcome based on treatment status
df['outcome'] = np.where(df['treatment'] == 1, df['Y1'], df['Y0'])

# Adding pre-treatment and post-treatment periods (assuming treatment starts at time 5 for everyone)
df['period_type'] = np.where(df['time_id'] < 5, 'pre-treatment', 'post-treatment')

In [4]:
df.head()

,unit_id,time_id,treatment,covariate1,covariate2,cohort,Y0,treatment_effect,Y1,outcome,period_type
0,1,1,0,0.738467,0.227460,2011,4.499240,2,6.499240,4.499240,pre-treatment
1,1,2,0,0.171368,1.307143,2012,5.053176,2,7.053176,5.053176,pre-treatment
2,1,3,0,-0.115648,-1.607483,2011,6.911211,2,8.911211,6.911211,pre-treatment
3,1,4,0,-0.301104,0.184634,2010,5.464753,2,7.464753,5.464753,pre-treatment
4,1,5,0,-1.478522,0.259883,2011,5.568139,2,7.568139,5.568139,post-treatment


In [5]:
# Apply the function
result_df = compute_treatment_effects(df, 'outcome', 'treatment', 'unit_id', 'time_id', covariates=['covariate1', 'covariate2'])
result_df.head()

Skipping column period_type as it appears to be truly textual.


,unit_id,time_id,treatment,covariate1,covariate2,cohort,Y0,treatment_effect,Y1,outcome,period_type,y_0,t_effects
0,1,1,0,0.738467,0.227460,2011,4.499240,2,6.499240,4.499240,pre-treatment,NaN,NaN
1,1,2,0,0.171368,1.307143,2012,5.053176,2,7.053176,5.053176,pre-treatment,NaN,NaN
2,1,3,0,-0.115648,-1.607483,2011,6.911211,2,8.911211,6.911211,pre-treatment,NaN,NaN
3,1,4,0,-0.301104,0.184634,2010,5.464753,2,7.464753,5.464753,pre-treatment,NaN,NaN
4,1,5,0,-1.478522,0.259883,2011,5.568139,2,7.568139,5.568139,post-treatment,NaN,NaN


In [6]:
result_df.loc[result_df['treatment']==1].head()

,unit_id,time_id,treatment,covariate1,covariate2,cohort,Y0,treatment_effect,Y1,outcome,period_type,y_0,t_effects
20,3,1,1,-0.479174,-0.974682,2010,6.312380,2,8.312380,8.312380,pre-treatment,6.312380,2.0
22,3,3,1,-1.106335,1.158596,2010,5.067553,2,7.067553,7.067553,pre-treatment,5.067553,2.0
23,3,4,1,-1.196207,-0.820682,2010,3.813685,2,5.813685,5.813685,pre-treatment,3.813685,2.0
24,3,5,1,0.812526,0.963376,2012,6.469303,2,8.469303,8.469303,post-treatment,6.469303,2.0
25,3,6,1,1.356240,0.412781,2012,10.255939,2,12.255939,12.255939,post-treatment,10.255939,2.0
